In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-31"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
95,2024-08-31,América Do Sul Torneio Sul,18:30,Brasil F,Argentina F,1.67,2.02,125.5,1.80,1.86,-3.5,2.05,1.65,EcmhxF5j,0.598802,0.495050,0.555556,0.537634,0.093852,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.134139,0.023184,0.152888,0.00,0.00,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
96,2024-08-31,América Do Sul Torneio Sul,21:00,Chile F,Uruguai F,1.25,3.43,123.5,1.80,1.86,-9.5,1.94,1.73,nPQYlhkd,0.800000,0.291545,0.555556,0.537634,0.091545,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.658758,0.023184,0.080922,0.00,0.00,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
97,2024-08-31,Eua Wnba,16:00,Washington Mystics F,Connecticut Sun F,3.75,1.23,152.5,1.81,2.00,7.5,2.00,1.85,KCRaLCuB,0.266667,0.813008,0.552486,0.500000,0.079675,0.8,0.6,NaN,NaN,228.100,137.570162,0.603113,141.394,43.859475,0.310193,240.552,141.394,210.54,124.80,0.0,0.0,0.0,0.0,0.715626,0.070525,0.055099,-2.85,-0.57,-4.824561,0.0,0.0,0.0,-0.71,-0.142,-1.619718,0.0,0.0,0.0
98,2024-08-31,Filipinas Copa Dos Governadores,07:00,San Miguel Beermen,NLEX Road Warriors,1.23,3.97,214.5,1.86,1.88,-9.5,1.94,1.77,WWtHNipA,0.813008,0.251889,0.537634,0.531915,0.064897,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,176.00,0.0,0.0,0.0,0.0,0.745182,0.007563,0.064802,0.00,0.00,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
99,2024-08-31,Mundo Amistoso Interclubes,06:00,Çağdaş Bodrum Spor (Tur),Gaziantep (Tur),9.35,1.04,143.5,1.81,1.87,15.5,1.97,1.71,bP5McNLE,0.106952,0.961538,0.552486,0.534759,0.068490,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,157.95,0.0,0.0,0.0,0.0,1.131099,0.023058,0.099917,0.00,0.00,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
100,2024-08-31,Mundo Amistoso Interclubes,08:30,OGM Ormanspor (Tur),Ankara Kolejliler (Tur),2.65,1.45,144.5,1.88,1.78,4.5,1.95,1.72,fiaVe1jR,0.377358,0.689655,0.531915,0.561798,0.067014,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,111.80,0.00,0.0,0.0,0.0,0.0,0.413916,0.038640,0.088629,0.00,0.00,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
101,2024-08-31,Mundo Amistoso Interclubes,08:30,Zhejiang Guangsha (Chn),Basquet Girona (Esp),5.61,1.09,154.5,1.83,1.83,11.5,1.95,1.72,O6k1juEl,0.178253,0.917431,0.546448,0.546448,0.095684,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,112.88,0.0,0.0,0.0,0.0,0.954066,0.000000,0.088629,0.00,0.00,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
102,2024-08-31,Mundo Amistoso Interclubes,11:30,Paralimni (Cyp),Keravnos S. (Cyp),8.30,1.03,151.5,1.83,1.83,16.5,2.01,1.68,pMwAlJr1,0.120482,0.970874,0.546448,0.546448,0.091356,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,1.101965,0.000000,0.126474,0.00,0.00,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
103,2024-08-31,Mundo Amistoso Interclubes,13:00,Nanterre (Fra),Cholet (Fra),1.62,2.10,161.5,1.83,1.83,-3.5,1.98,1.70,dGeb2ecR,0.617284,0.476190,0.546448,0.546448,0.093474,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.182479,0.000000,0.107603,0.00,0.00,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
104,2024-08-31,Mundo Amistoso Interclubes,15:00,Manresa (Esp),Valencia (Esp),3.07,1.30,165.5,1.88,1.78,5.5,2.03,1.67,hnsUdZKl,0.325733,0.769231,0.531915,0.561798,0.094964,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,204.00,0.00,0.0,0.0,0.0,0.0,0.572805,0.038640,0.137599,0.00,0.00,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
